In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
%matplotlib inline

In [2]:
#Team Based Table
attendance_df = pd.read_csv("/Users/jenniferwu/Documents/NBA_Social_Influence/dataset/nba_2017_attendance.csv")
endorsement_df = pd.read_csv("/Users/jenniferwu/Documents/NBA_Social_Influence/dataset/nba_2017_endorsements.csv")
valuations_df = pd.read_csv("/Users/jenniferwu/Documents/NBA_Social_Influence/dataset/nba_2017_team_valuations.csv")
elo_df = pd.read_csv("/Users/jenniferwu/Documents/NBA_Social_Influence/dataset/nba_2017_elo.csv")


#PLayer Based Table
salary_df = pd.read_csv("/Users/jenniferwu/Documents/NBA_Social_Influence/dataset/nba_2017_salary.csv")
pie_df = pd.read_csv("/Users/jenniferwu/Documents/NBA_Social_Influence/dataset/nba_2017_pie.csv")
plus_minus_df = pd.read_csv("/Users/jenniferwu/Documents/NBA_Social_Influence/dataset/nba_2017_real_plus_minus.csv")
br_stats_df = pd.read_csv("/Users/jenniferwu/Documents/NBA_Social_Influence/dataset/nba_2017_br.csv")
nba_2017_twitter_players = pd.read_csv("/Users/jenniferwu/Documents/NBA_Social_Influence/dataset/nba_2017_twitter_players.csv")
nba_2017_player_wikipedia = pd.read_csv("/Users/jenniferwu/Documents/NBA_Social_Influence/dataset/nba_2017_player_wikipedia.csv")

In [191]:
print(attendance_df.shape)
print(endorsement_df.shape)
print(valuations_df.shape)
print(elo_df.shape)

print(salary_df.shape)
print(pie_df.shape)
print(plus_minus_df.shape)
print(br_stats_df.shape)
print(nba_2017_twitter_players.shape)
print(nba_2017_player_wikipedia.shape)

(30, 5)
(10, 4)
(30, 2)
(30, 3)
(449, 4)
(486, 22)
(468, 8)
(486, 30)
(329, 3)
(162817, 5)


In [192]:
print(salary_df.columns,"\n")
print(pie_df.columns,"\n")
print(plus_minus_df.columns,"\n")
print(br_stats_df.columns,"\n")
print(nba_2017_twitter_players.columns,"\n")
print(nba_2017_player_wikipedia.columns,"\n")

Index(['NAME', 'POSITION', 'TEAM', 'SALARY'], dtype='object') 

Index(['PLAYER', 'TEAM', 'AGE', 'GP', 'W', 'L', 'MIN', 'OFFRTG', 'DEFRTG',
       'NETRTG', 'AST%', 'AST/TO', 'AST RATIO', 'OREB%', 'DREB%', 'REB%',
       'TO RATIO', 'EFG%', 'TS%', 'USG%', 'PACE', 'PIE'],
      dtype='object') 

Index(['NAME', 'TEAM', 'GP', 'MPG', 'ORPM', 'DRPM', 'RPM', 'WINS'], dtype='object') 

Index(['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PS/G'],
      dtype='object') 

Index(['PLAYER', 'TWITTER_FAVORITE_COUNT', 'TWITTER_RETWEET_COUNT'], dtype='object') 

Index(['Unnamed: 0', 'names', 'pageviews', 'timestamps', 'wikipedia_handles'], dtype='object') 



In [193]:
salary_df.rename(columns={'NAME': 'PLAYER'}, inplace=True)
salary_df.drop(["POSITION","TEAM"], inplace=True, axis=1)

In [194]:
plus_minus_df.rename(columns={"NAME":"PLAYER"}, inplace=True)
players = []
for player in plus_minus_df["PLAYER"]:
    plyr, _ = player.split(",")
    players.append(plyr)
plus_minus_df.drop(["PLAYER"], inplace=True, axis=1)
plus_minus_df["PLAYER"] = players

In [195]:
pie_df_subset = pie_df[["PLAYER", "PIE", "PACE"]].copy()

In [196]:
nba_players_df = br_stats_df.copy()
nba_players_df.rename(columns={'Player': 'PLAYER','Pos':'POSITION', 'Tm': "TEAM", 'Age': 'AGE'}, inplace=True)
nba_players_df.drop(["G", "GS", "TEAM"], inplace=True, axis=1)

In [197]:
nba_2017_player_wikipedia['pageview_mean']=nba_2017_player_wikipedia.pageviews
nba_2017_player_wikipedia['pageview_sum']=nba_2017_player_wikipedia.pageviews
nba_2017_player_wikipedia['PLAYER']=nba_2017_player_wikipedia.names
nba_2017_player_wikipedia = nba_2017_player_wikipedia.groupby(['PLAYER'], as_index = False).agg({'pageview_sum':"sum",
                                                                    'pageview_mean':"mean"})

In [205]:
nba_2017_twitter_players.head()

,PLAYER,TWITTER_FAVORITE_COUNT,TWITTER_RETWEET_COUNT
0,Russell Westbrook,2130.5,559.0
1,James Harden,969.0,321.5
2,Isaiah Thomas,467.5,155.5
3,Anthony Davis,368.0,104.0
4,DeMar DeRozan,0.0,186.0


## START THE JOIN

In [198]:
nba_players_df = nba_players_df.merge(nba_2017_player_wikipedia, how = 'inner', on = 'PLAYER')
nba_players_df.shape

(455, 29)

In [199]:
nba_players_df = nba_players_df.merge(pie_df_subset, how = 'inner', on = 'PLAYER')
nba_players_df.shape

(437, 31)

In [200]:
nba_players_df = nba_players_df.merge(salary_df, how = 'left', on = 'PLAYER')
nba_players_df.shape

(443, 32)

In [201]:
nba_players_df = nba_players_df.merge(plus_minus_df, how = 'inner', on = 'PLAYER')
nba_players_df.shape

(428, 39)

In [206]:
nba_players_df = nba_players_df.merge(nba_2017_twitter_players, how = 'left', on = 'PLAYER')
nba_players_df.shape

(428, 41)

In [207]:
#checking for duplicates
nba_players_df[nba_players_df.duplicated()]

,Rk,PLAYER,POSITION,AGE,MP,FG,FGA,FG%,3P,3PA,...,SALARY,TEAM,GP,MPG,ORPM,DRPM,RPM,WINS,TWITTER_FAVORITE_COUNT,TWITTER_RETWEET_COUNT


In [209]:
#NA columns 
nba_players_df.isna().sum()[nba_players_df.isna().sum() > 0 ]

3P%                        29
FT%                         7
SALARY                    102
TWITTER_FAVORITE_COUNT    139
TWITTER_RETWEET_COUNT     139
dtype: int64

In [213]:
nba_players_df.to_csv('/Users/jenniferwu/Documents/NBA_Social_Influence/final_master_data_file.csv')